In [27]:
from __future__ import print_function
%matplotlib inline
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

Using Theano backend.


In [18]:
sample_rate_down = 1
hop_length_down = 8
sr = 11025 * 16 / sample_rate_down
hop_length = 512 / (sample_rate_down * hop_length_down)
samples_per_beat = 24 / 4
steps_per_bar = 24
class SongFile:
    # misc includes
    # - offset
    # - bpm
    def load_misc_from_music(self, y):
        _, beat_frames = librosa.beat.beat_track(y=y, sr=sr, hop_length=hop_length)
        beat_times = librosa.frames_to_time(beat_frames, sr=sr, hop_length=hop_length)
        self.offset = beat_times[0]
        self.bpm = get_beats(beat_times, beat_frames)

    def load_misc_from_stepfile(self):
        with open(self.stepfile, "r") as txt:
            step_file = txt.read()
            step_file = step_file.replace('\n', 'n')
            bpm_search = re.search('#BPMS:([0-9.=,]*);', step_file)
            bpm_string = bpm_search.group(1)
            self.bpm = float(bpm_string.split('=')[1]) if len(bpm_string.split(',')) == 1 else 0
            
            offset_search = re.search('#OFFSET:([0-9.-]*);', step_file)
            self.offset = -float(offset_search.group(1))

    def calculate_indices(self, y):
        # take samples_per_beat samples for each beat (need 3rds, 8ths)
        seconds = len(y) / sr
        num_samples = int(seconds * samples_per_beat * self.bpm / 60)
        beat_length = 60. / self.bpm
        sample_length = beat_length / samples_per_beat
        
        if self.offset < 0:
            self.offset += 4 * beat_length
        
        sample_times = [self.offset + (sample_length * i) for i in range(num_samples)]
        # only take samples where music still playing
        self.indices = [round(time * sr) for time in sample_times if round(time * sr) < len(y)]
        
    def calculate_features(self, y):
        y_harmonic, y_percussive = librosa.effects.hpss(y)
        beat_frames = librosa.samples_to_frames(self.indices)

        # Compute MFCC features from the raw signal
        mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

        # And the first-order differences (delta features)
        mfcc_delta = librosa.feature.delta(mfcc)

        # Stack and synchronize between beat events
        # This time, we'll use the mean value (default) instead of median
        beat_mfcc_delta = librosa.feature.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

        # Compute chroma features from the harmonic signal
        chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

        # Aggregate chroma features between beat events
        # We'll use the median value of each feature between beat frames
        beat_chroma = librosa.feature.sync(chromagram, beat_frames, aggregate=np.median)

        custom_hop = 256
        onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=custom_hop)
        onsets = librosa.onset.onset_detect(y=y, sr=sr, onset_envelope=onset_env, hop_length=custom_hop)

        i = 0
        onset_happened_in_frame = [0] * (len(self.indices) + 1)
        for onset in onsets:
            onset_scaled = onset * custom_hop
            while abs(onset_scaled - self.indices[i]) > abs(onset_scaled - self.indices[i + 1]):
                i += 1
            onset_happened_in_frame[i] = max(onset_env[onset], onset_env[onset + 1], onset_env[onset + 2], onset_env[onset + 3], onset_env[onset + 4])

        indices = [0]
        indices.extend(self.indices)
        max_offset_bounds = [(int(indices[i] / custom_hop), int(indices[i + 1] / custom_hop)) for i in range(len(indices) - 1)]
        max_offset_strengths = [max(onset_env[bounds[0]:bounds[1]]) for bounds in max_offset_bounds]
        max_offset_strengths.append(0)

        # Finally, stack all beat-synchronous features together
        self.beat_features = np.vstack([beat_chroma, beat_mfcc_delta, [onset_happened_in_frame, max_offset_strengths]])

    def __init__(self, key, folder, stepfile, music_file, load_type):
        key = key
        self.folder = folder
        self.stepfile = stepfile
        self.music_file = music_file
        y = None
        print ('Loading song {0}'.format(key))
        
        if load_type == 'from_music' or load_type == 'from_stepfile':
            if load_type == 'from_music':
                print ('Loading music')
                y, _ = librosa.load(self.music_file, sr=sr)
                print ('Calculating misc from music')
                self.load_misc_from_music(y)
            else:
                print ('Loading misc from stepfile')
                self.load_misc_from_stepfile()
                if self.bpm == 0:
                    raise Exception('Inconsistent bpm')
                print ('Loading music')
                y, _ = librosa.load(self.music_file, sr=sr)
                

            print ('Calculating indices')
            self.calculate_indices(y)
            print ('Calculating features')
            self.calculate_features(y)
            print ('Saving song\n')
            pd.DataFrame([self.offset, self.bpm]).to_csv('data/{0}_misc.csv'.format(key), index=False)
            pd.DataFrame(self.beat_features).to_csv('data/{0}_beat_features.csv'.format(key), index=False)
            
        if load_type == 'from_store':
            if not '{0}_beat_features.csv'.format(key) in listdir('data'):
                print ('Song hasnt been loaded yet')
            else:
                [self.offset], [self.bpm] = pd.read_csv('data/{0}_misc.csv'.format(key)).values
                self.beat_features = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values


# # Some useful functions to load induvidual or lists of songs
# - load_song(pack: String, pack: String, force_new: Bool)
# - load_songs(songs: Array(Pair(String~pack, String~title)), force_new: Bool)
# - load_all_songs(force_new: Bool)

# In[4]:

def load_songs(songs, load_type):
    return {'{0}~{1}'.format(song[0], song[1]): SongFile(song[0], song[1], load_type) for song in songs}

def load_all_songs(load_type):
    songs = [('In The Groove', song) for song in listdir('StepMania/Songs/In The Groove') if song != '.DS_Store']
    songs.extend([('a_test', song) for song in ['A', 'B', 'C']])
    return load_songs(songs, load_type)


# # Functions to get bpm from song
# - get_beats(beat_times: Array(Float), beat_frames: Array(Int))

# In[5]:

def get_beats(beat_times, beat_frames):
    changes = []
    changes_time = []
    for i in range(len(beat_frames) - 1):
        changes.append(beat_frames[i + 1] - beat_frames[i])
        changes_time.append(beat_times[i + 1] - beat_times[i])

    sorted_changes = sorted(changes)
    median = sorted_changes[int(len(changes) / 2)]
    median = max(set(sorted_changes), key=sorted_changes.count)

    changes_counted = [False] * len(changes)
    time_changes_sum = 0
    time_changes_count = 0
    for i in range(len(changes)):
        # can use other factors (eg if song has a slow part take double beats into accout)
        # in [0.5, 1, 2]:
        for change_factor in [1]:
            if abs((changes[i] * change_factor) - median) <= hop_length_down:
                changes_counted[i] = True
                time_changes_sum += (changes_time[i] * change_factor)
                time_changes_count += change_factor
            
    average = time_changes_sum / time_changes_count
    
    time_differences = []
    earliest_proper_beat = 1
    for i in range(1, len(beat_times) - 1):
        if changes_counted[i] & changes_counted[i - 1]:
            earliest_proper_beat = i
            break
            
    last_proper_beat = len(beat_times) -2
    for i in range(1, len(beat_times) - 1):
        if changes_counted[len(beat_times) - i - 1] & changes_counted[len(beat_times) - i - 2]:
            last_proper_beat = len(beat_times) - i - 1
            break
    
    time_differences = []
    buffer = 5
    for i in range(20):
        start_beat = earliest_proper_beat + buffer * i
        if changes_counted[start_beat] & changes_counted[start_beat - 1]:
            for j in range(20):
                end_beat = last_proper_beat - buffer * j
                if changes_counted[end_beat] & changes_counted[end_beat - 1]:
                    time_differences.append(beat_times[end_beat] - beat_times[start_beat])
        
    # get num beats, round, and make new average
    new_averages = [time_difference / round(time_difference / average) for time_difference in time_differences]
    #print (new_averages)
    new_averages.sort()
    num_averages = len(new_averages)
    #new_average = sum(new_averages[5:num_averages - 5]) / (num_averages - 10)
    new_average = new_averages[int(num_averages/2)]
    bpm = 60./new_average
    while bpm >= 200:
        bpm /= 2
    while bpm < 100:
        bpm *= 2
    return bpm

In [23]:
def get_song_features(key):
    pack, song = key.split('~')
    folder = 'StepMania/Songs/{0}/{1}/'.format(pack, song)
    stepfiles = [file for file in listdir(folder) if file.endswith('.ssc') or file.endswith('.sm')]
    musicfiles = [file for file in listdir(folder) if file.endswith('.ogg') or file.endswith('.mp3')]
    stepfile = folder + stepfiles[0]
    music = folder + musicfiles[0]


    song_data = [key, folder, stepfile, music]
    
    if '{0}_beat_features.csv'.format(song_data[0]) in listdir('data'):
        print ('Song Already Loaded')
    else:
        SongFile(song_data[0], song_data[1], song_data[2], song_data[3], 'from_stepfile')

    gc.collect()

In [25]:
songs = ['Anubis', 'Bend Your Mind', 'Boogie Down', 'Bouff', 'Bubble Dancer']
[get_song_features('a_test~{0}'.format(song)) for song in songs]

Song Already Loaded
Song Already Loaded
Loading song a_test~Boogie Down
Loading misc from stepfile
Loading music
Calculating indices
Calculating features


/Users/jgzuke/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:482: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return array(a, dtype, copy=False, order=order)


Saving song

Loading song a_test~Bouff
Loading misc from stepfile
Loading music
Calculating indices
Calculating features


/Users/jgzuke/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:482: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return array(a, dtype, copy=False, order=order)


Saving song

Loading song a_test~Bubble Dancer
Loading misc from stepfile
Loading music
Calculating indices
Calculating features


/Users/jgzuke/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:482: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return array(a, dtype, copy=False, order=order)


Saving song



[None, None, None, None, None]

In [29]:
samples_back_included = 8
num_classes = 5
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return (0, 0)
    return notes[index][0].count('1')
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 20) != 0) and not (row_y == 1 and random.randint(0, 3) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

def build_batch_generator():
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use:
        yield (get_features_for_song(song_data[0]))

# Total 243 songs
def build_training_data(songs_start, songs_end, is_full = False):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use[songs_start:songs_end]:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return X, y

def calculate_importance(row):
    return (1 - row[0]) * (row[1] + row[2] * 2 + row[3] * 30 + row[4] * 40)

def step_song(key, clf):
    song_X, song_y = get_features_for_song(key, True)
    new_song_y = clf.predict(song_X)
    beat_importance = [calculate_importance(row) for row in new_song_y]
    
    #print ('Length: ' + str(len(new_song_y)))
    #plt.plot([new_song_y[i] for i in range(len(new_song_y)) if i % 12 == 0])
    #plt.show()
    
    pd.DataFrame(beat_importance).to_csv('generated_data/{0}_importance_generated.csv'.format(key), index=False)

In [30]:
beat_feature_model = load_model('models/beat_importance_model.h5')

In [31]:
keys = ['a_test~{0}'.format(song) for song in ['Anubis', 'Bend Your Mind', 'Boogie Down', 'Bouff', 'Bubble Dancer']]
[step_song(key, beat_feature_model) for key in keys]

Exception: Error when checking : expected dense_input_2 to have shape (None, 324) but got array with shape (0, 1)